In [ ]:
%pylab inline
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, Variable
import matplotlib.patches as mpatches
import cartopy
from datetime import timedelta as delta
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import xarray as xr
from os import environ
import matplotlib.animation as animation
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

In [ ]:
# exp = 'china'
exp = 'americas'
# exp = 'bwd'

In [ ]:
galapagos_extent = [-91.8, -89, -1.4, 0.7]
environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

# pfile = xr.open_dataset('galapagosparticles_fwd_2yrs.nc', decode_cf=True)
pfile = xr.open_dataset('/scratch/evansebille/galapagosparticles_%s.nc' %exp, decode_cf=True)
lon = np.ma.filled(pfile.variables['lon'], np.nan)
lat = np.ma.filled(pfile.variables['lat'], np.nan)
time = np.ma.filled(pfile.variables['time'], np.nan)
age = np.ma.filled(pfile.variables['age'], np.nan)

if 'bwd' not in exp:
    visitedgalapagos = np.ma.filled(pfile.variables['visitedgalapagos'], np.nan)
else:
    visitedgalapagos = np.ones(lat.shape)
pfile.close()

if 'bwd' not in exp:
    visitedtime = np.nan*np.zeros(visitedgalapagos.shape[0])
    for p in range(visitedgalapagos.shape[0]):
        if np.any(visitedgalapagos[p, :] == 1):
            I = np.where(visitedgalapagos[p, :] == 1)[0][0]
            visitedtime[p] = age[p, I]/86400.
            visitedgalapagos[p, :] = 1
    nvisited = len(np.where(visitedgalapagos[:, 0]==1)[0])
        
nsites = np.unique(lat[:, 0]).shape[0]
startsite = np.nan*np.zeros(lat.shape[0])
startcountry = np.nan*np.zeros(lat.shape[0])
for s in range(nsites):
    I = np.where(lat[:, 0] == lat[s, 0])
    startsite[I] = s
    if exp == 'americas':
        countrylats = [-60, -18, -3, 1.5, 7.3, 8.1, 11.0, 12.8, 13.7, 14.6, 32.5]
#         countrynames = ['CH', 'PE', 'EC', 'CO', 'PA', 'CR', 'NI', 'SV', 'GT', 'MX']
        countrynames = ['Chile', 'Peru', 'Ecuador', 'Colombia', 'PA', 'CR', 'NI', 'SV', 'GT', 'Mexico']
        C = np.where(lat[s, 0] > countrylats)[0]
        startcountry[I] = C[-1]

pfile = xr.open_dataset('inino5.nc', decode_cf=False)
ninotime = np.arange(1856, 2019, 1./12)
nino34 = pfile['Nino3.4'][:]
pfile.close()

In [ ]:
from matplotlib import colors
pylab.rcParams['figure.figsize'] = (10, 7)

projection = cartopy.crs.PlateCarree(central_longitude=180)
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': projection})
ax.coastlines()
ax.add_feature(cartopy.feature.LAND)

if exp == 'americas':
    bins = [np.arange(-180, -60, 1), np.arange(-60, 45, 1)]
    recfc = None
elif exp == 'china':
    bins = [np.arange(90, 200, 1), np.arange(-20, 50, 1)]
#     axextent = [-90, 120, -20, 50] 
    recfc = None
elif 'bwd' in exp:
    bins = [np.arange(-180, -60, 1), np.arange(-60, 45, 1)]
    recfc = 'red'


ax.add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]], width=galapagos_extent[1]-galapagos_extent[0], 
                                height=galapagos_extent[3]-galapagos_extent[2], facecolor=recfc, alpha=1, linewidth=1,
                                edgecolor='k', transform=cartopy.crs.PlateCarree()))

h2d = ax.hist2d(lon[~np.isnan(lon)], lat[~np.isnan(lat)], bins=bins, norm=colors.LogNorm(), transform=cartopy.crs.PlateCarree())
ax.set_extent([bins[0][0], bins[0][-1], bins[1][0], bins[1][-1]], crs=cartopy.crs.PlateCarree())
plt.colorbar(h2d[-1])

plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 7)

years = range(2000, 2011)
yrs = time[:, 0].astype('datetime64[Y]').astype('int')+1970
countriestoplot = [1, 2, 3]
fracperyear = np.zeros((len(countriestoplot), len(years)))
for c, country in enumerate(countriestoplot):
    for y in range(len(years)):
        I = np.where(np.logical_and(yrs == years[y], startcountry == country))[0]
        J = np.where(np.isfinite(visitedtime[I]))[0]
        fracperyear[c, y] = 100.0*len(J)/len(I)

fig, ax = plt.subplots(1, 1)
lineObjects = ax.plot([y+0.5 for y in years], fracperyear.T, 'o-')
plt.legend(iter(lineObjects), [countrynames[c] for c in countriestoplot])
plt.xticks(years)
plt.xlabel('Year')
ax.set_ylabel('Fraction of particles that pass through Galapagos [%]')

yl = ax.get_ylim()
for elnino in [2003, 2005, 2007]:
    ax.add_collection(PatchCollection([Rectangle((elnino-1./12, yl[0]), 3./12, yl[1]-yl[0])], 
                      facecolor='r', alpha=0.2, edgecolor=None))
    ax.text(elnino+0.5/12., yl[1]+0.1, 'EL', horizontalalignment='center')

for lanina in [2000, 2001, 2008, 2009, 2010]:
    ax.add_collection(PatchCollection([Rectangle((lanina-1./12, yl[0]), 3./12, yl[1]-yl[0])], 
                      facecolor='b', alpha=0.2, edgecolor=None))
    ax.text(lanina+0.5/12., yl[1]+0.1, 'LN', horizontalalignment='center')
ax.set_ylim([0, yl[1]])
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 7)

if np.any(np.isfinite(visitedtime)):
    plt.hist(visitedtime[np.isfinite(visitedtime)], np.arange(0, np.nanmax(visitedtime), 10))
    plt.gca().set_xlim([0, np.nanmax(visitedtime)])
    plt.xlabel('Time to first pass through Galapagos [days]')
    plt.ylabel('Number of particles')
    plt.title('%d of %d (%.1f%%) particles reach Galapagos' % (nvisited, visitedgalapagos.shape[0],
          100.0*nvisited/visitedgalapagos.shape[0]))
    plt.show()

In [ ]:
pct_visitgalapagos = np.zeros(nsites)
startlat = np.zeros(nsites)
for s in range(nsites):
    I = np.where(startsite==s)[0]
    nvisit = len(np.where(visitedgalapagos[I,0]==1)[0])
    pct_visitgalapagos[s] = 100*nvisit/len(I)
    startlat[s] = lat[I[0], 0]

plt.plot(startlat, pct_visitgalapagos,'.-')
plt.xlabel('Starting latitude [degrees N]')
plt.ylabel('Fraction of particles passing through Galapagos [%]')
ylim = plt.gca().get_ylim()
xlim = [-25, 25] # plt.gca().get_xlim()
for s in range(1, len(countrylats)):
    plt.plot([countrylats[s], countrylats[s]], ylim, 'k:')
    xmin = max(xlim[0], countrylats[s-1])
    xmax = min(xlim[1], countrylats[s])
    plt.text((xmin+xmax)/2, ylim[1]-3, countrynames[s-1], horizontalalignment='center')

plt.gca().set_ylim(ylim)
plt.gca().set_xlim(xlim)
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 7)

projection = cartopy.crs.PlateCarree(central_longitude=180)
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': projection})
ax.coastlines()
ax.add_feature(cartopy.feature.LAND)

ax.add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]],
                                width=galapagos_extent[1]-galapagos_extent[0], 
                                height=galapagos_extent[3]-galapagos_extent[2], facecolor='red', alpha=0.2,
                                transform=cartopy.crs.PlateCarree()))

if exp == 'americas':
    axextent = [0, 120, -15, 15]
elif exp == 'china':
    axextent = [-90, 120, -20, 50] 
elif 'bwd' in exp:
    axextent = [0, 120, -45, 20] 
plottimes = np.arange(np.min(time), np.max(time), dtype='datetime64[5D]')

b = np.where(np.logical_and(time >= plottimes[0], time < plottimes[1]))
scat = ax.scatter(lon[b], lat[b], s=3, c=visitedgalapagos[b], transform=cartopy.crs.Geodetic(), cmap="bwr")
ax.set_extent(axextent, crs=projection)
scat.set_clim(0, 1)

ttl = ax.set_title('Particles on ' + str(plottimes[0]) + ' (red particles pass through Galapagos)')
frames = np.arange(0, len(plottimes)-1)

def animate(t):
    b = np.where(np.logical_and(time >= plottimes[t], time < plottimes[t+1]))
    scat.set_offsets(np.vstack((lon[b], lat[b])).transpose())
    scat.set_array(visitedgalapagos[b])
    ttl.set_text('Particles on ' + str(plottimes[t]) + ' (red particles pass through Galapagos)')
    return scat,

ax.set_extent(axextent, crs=projection)
anim = animation.FuncAnimation(fig, animate, frames=frames, interval=100, blit=True)
anim
anim.save('galapagosparticles_%s.gif' % exp, writer='imagemagick', fps=10)